In [17]:
# -*- coding: utf-8 -*-
import os
import sys
import math
import re
import pickle
import codecs
import numpy as np
import tensorflow as tf
from glob import glob
from tensorflow.models.rnn import rnn, rnn_cell
from tensorflow.python.framework import ops

In [7]:
epoch = 15 # epoch to train, 其实这个是指总的循环次数吗?
word_embed_dim = 650 # dimension of word embedding matrix 词向量矩阵
char_embed_dim = 15  # dimension of char embedding matrix 字向量矩阵
max_word_lenth = 65  # 词的最长长度, 这个对于中文没有必要好像...
batch_size = 100     # batch 最小的批处理
seq_length = 35      # no. of timesteps to unroll for ???
learning_Rate = 1.0
decay = 0.5  # decay of SGD
dropout_prob = 0.5   # probability of dropout layer, dropout layer到底是用在哪儿的???
feature_maps = [50,100,150,200,200,200,200] # no. of feature maps in CNN ???
kernels = [1,2,3,4,5,6,7]  # The width of CNN kernels ??? CNN的kernel 到底是啥???
#model = 'LSTMTDNN' # type of model to train and test, LSTM/LSTMTDNN
# data_dir
# dataset
# checkpoint_dir
forward_only = False # false for training
use_char = True  # use 字-lv 语言模型
use_word = False # use 词-lv 语言模型

In [8]:
# 写入
def save(fname, obj):
    with open(fname, 'w') as f:
        pickle.dump(obj, f)

# 读取
def load(fname):
    with open(fname, 'r') as f:
        pickle.load(f)

In [9]:
train_fname = 'data/wxb_gt_train.txt' # 黄金时代-> train
valid_fname = 'data/wxb_s_valid.txt'  # 沉默的大多数-> valid
test_fname = 'data/wxb_sf_test.txt'   # 思维的乐趣-> test
input_fnames = [train_fname, valid_fname, test_fname]

# vocab_fname = 'vocab.pkl'
# tensor_fname = 'data.pkl'
# char_fname = 'data_char.pkl'

In [21]:
# 处理 txt, 去掉空格空行
counts = []
for input_file in input_fnames:
    with codecs.open(input_file,'rw','utf-8') as f: # 用 codecs 打开 file 才不会出现编码问题
        a = f.read()
        a = a.replace(' ','') # 去掉空格
        a = a.replace('\n','') # 去掉空行
        print input_file,' with characters ',len(a) # 每个 txt 中有多少字
        # print a[:20]
        print input_file,'with unique characters ',len(set(a))
        #print 'sample of characters: ',list(set(a))[:10]

data/wxb_gt_train.txt  with characters  32443
data/wxb_gt_train.txt with unique characters  1829
data/wxb_s_valid.txt  with characters  9069
data/wxb_s_valid.txt with unique characters  1059
data/wxb_sf_test.txt  with characters  6215
data/wxb_sf_test.txt with unique characters  867


常用字: 一级常用字为3500, 二级常用字为3000字, 三级为姓氏人名地名等专有名词1605字, 总共8105字, http://www.gov.cn/gzdt/att/att/site1/20130819/tygfhzb.pdf

使用 HSK 字典, 总字数为2500常用字
https://github.com/OpenMindClub/OMAIWriter/wiki/ResDictionaries